# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> Tuple:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return `None`.


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

## solve_game

In order to solve the game, we will use the `solve_game` function. In order to process the information and find stronly and weakly dominate strategies, I have written a series of functions. I have provided below documentation and unit tests for all of them. In order to test the strategies, we will use the following example:

| Player 1 / Player 2  | 0 | 1 |
|----|----|----|
|0  | -6,-6 | 0,-10 |
|1  | -10,0 | -1,-1 | 

The overall strategy to solve the problem is the following. For each iteration of the game:
1. Define the strategies for each player in a format that is easily parsed. This is handled by `find_strategies`.
2. Find any dominated strategies (weakly or strongly) from those available for a given player. This is handeled by the  `find_weak_strategy()` function.
3. If found, remove any weak strategies from the game. This action is performed by the `remove_weak_strategy()` function.

The function `solve_game()` is a wrapper function which iterates over different states of the game performing these three actions until a single position is found or until the game is found to be stuck.

In [3]:
example = [[(-6,-6),(0,-10)],[(-10,0),(-1,-1)]]
from copy import deepcopy

### Finding strategies
`find_strategies()` is a function used to parse the available strategies for each player. The function takes in the current version of the game and stores the available sets of strategies in nested distionaries.

In [4]:
def find_strategies(game: list[list[tuple]]):
    game_shape = len(game),len(game[0])
    strategies = {}
    p0_strategies = {i:[] for i in range(len(game))}
    p1_strategies = {i:[] for i in range(len(game[0]))}
    #return p0_strategies, p1_strategies
    for i in range(len(game)):
        p0_strategy = []
        p1_strategy = []
        for j in range(len(game[i])):
            p0_strategies[i].append(game[i][j][0])
            p1_strategies[j].append(game[i][j][1])      
    strategies[0] = p0_strategies
    strategies[1] = p1_strategies
    return strategies          

In [5]:
assert find_strategies(example) == {0: {0: [-6, 0], 1: [-10, -1]}, 1: {0: [-6, 0], 1: [-10, -1]}}
assert find_strategies(prisoners_dilemma) == {0: {0: [-5, -1], 1: [-10, -2]}, 1: {0: [-5, -1], 1: [-10, -2]}}

### Finding weak strategies
The creationg of a more friendly data structure allows us to easily compare the available actions component wise, this while keeping track of the number of actions within a choice that is stronger. We also pass an indicator for us of weak or strong dominance, the differences between the two are:
1. Strong dominance requires the use of '>' and it must be the case that all the elements of a particular choice must be stronger than those of the weaker choice.
2. Weak dominance requires the use of '>=', where one of the strategies is strictly greater ('>') while the rest can be greater than or equal ('>='). This means that the player has the ability to improve but not to reduce its utility. 

The function `find_weak_strategy()` performs this actions for a particular player. `find_weak_strategy()` the loops over all the available actions, comparing each one agains each other action available. Whenever an action is found to be weaker than the one it is compared against, the function returns a tuple of the player and the weaker action index.

In [6]:
def find_weak_strategy(strategies: dict, weak_dominance:bool, player:int):
    player_strategies = deepcopy(strategies[player])
    number_components = len(player_strategies[0])
    
    for i in range(len(player_strategies)):
        strategies = list(player_strategies.keys())
        strategies_to_evaluate = [strat for strat in strategies if strat != i]
        for j in strategies_to_evaluate:
            valid_components = 0
            strict_valid_components = 0
            non_strict_valid_components = 0
            for x in range(len(player_strategies[i])):
                if weak_dominance:
                    if player_strategies[i][x] > player_strategies[j][x]:
                        strict_valid_components += 1
                    elif player_strategies[i][x] >= player_strategies[j][x]:
                        non_strict_valid_components += 1
                    if strict_valid_components>=1 and (strict_valid_components+non_strict_valid_components) == number_components:
                        # print(f"""Player {player} found a weakly dominated strategy""")
                        return player, j                    
                else:
                    if player_strategies[i][x] > player_strategies[j][x]:
                        valid_components += 1
                    if valid_components == number_components:
                        # print(f"""Player {player} found a strongly dominated strategy""")
                        return player, j                    
    # print(f"""Player {player} did not find a {'weakly' if weak_dominance else 'strongly'} dominated strategy""")
    return None

In [7]:
strategies = find_strategies(example)
assert find_weak_strategy(strategies, False, 0) == (0,1)
assert find_weak_strategy(strategies, False, 1) == (1,1)

### Removing weak strategies
Up to here we have a mechanism to detect weak strategies. The function `remove_weak_strategy()` takes the current state of the game and returns a modified version of it. The function also takes as parameter a tuple of a strategy to remove containing the player and strategy. This is important because given the structure of the game, for player 0 we need to remove rows and for player 1 we would need to remove columns of the game. 

In [8]:
def remove_weak_strategy(game: list, strategy_to_remove: tuple) -> list:
    new_game = deepcopy(game)
    player = strategy_to_remove[0]
    weak_strategy = strategy_to_remove[1]
    if player == 0:
        new_game.pop(weak_strategy)
    else:
        for row in new_game:
            row.pop(weak_strategy)
    # print(f"""Player {strategy_to_remove[0]} removed a strategy from them game""")
    return new_game

In [9]:
to_remove = find_weak_strategy(strategies, False, 0) #(0,1)
to_remove2 = find_weak_strategy(strategies, False, 1) #(1,1)
# We should take the game and return a version of it in which for player 0 we remove strategy 1. This means we should get
# [[(-6, -6), (0, -10)]]
assert remove_weak_strategy(example, to_remove) == [[(-6, -6), (0, -10)]]
# In the second example we remove strategy 1 the selected strategy for player 1. This means we should get 
# [[(-6, -6)], [(-10, 0)]]
assert remove_weak_strategy(example, to_remove2) == [[(-6, -6)], [(-10, 0)]]

### Putting it all together
With all of the recipes in place to solve the problem, the function `solve_game()` iterates through the game implementing the 3 steps previously mentioned. In the implementation, we define a nested list with the available strategies/actions available to both players. As we loop trough the three functions sequentially, we keep track of the left over available strategies. The while loop is broken whenever:
1. there is only one strategy left available per player, or 
2. another way in which we brake off the loop is by letting is player go twice after the other and finding that there was no resulting improvement in the number of strategies available. When this happens, we say that the game is stuck and return None. 

In the case that a winning strategy is found for both players, the function returns a tuple with the coordinates of such strategies in the game. Notice that these coordinates corresponds to the nested lists we have been keeping track of. 

In [10]:
def solve_game(game: list[list[tuple]], weak:bool=False) -> tuple:
    # print(f"""\033[1mGame with {'weak' if weak else 'strong'} dominance\033[0;0m""")
    player0_strategies = [i for i in range(len(game))]
    player1_strategies = [i for i in range(len(game[0]))]
    countdown = [player0_strategies,player1_strategies]
    
    while len(countdown[0])>1 or len(countdown[1])>1:
        original_countdown = deepcopy(countdown)
        strategies = find_strategies(game)
        for i in [0,1,0,1]:
            # print(f"""\nPlayer {i}'s Turn:'""")
            weak_strategy = find_weak_strategy(strategies, weak, i)
            if weak_strategy is None:
                continue
            else:
                countdown[i].pop(weak_strategy[1])
                game = remove_weak_strategy(game, weak_strategy)
                # print(weak_strategy)
                strategies = find_strategies(game)
        if countdown == original_countdown:
            # print(f'The game is stuck\n')
            return None
    
    winning_strategy = (countdown[0][0],countdown[1][0])
    # print(f"""\nThe winning strategy is {winning_strategy}""")    
    
    return winning_strategy

In [11]:
# In the previous example we found that player 0 eliminates their strategy 1 and that player 1 also eliminates their strategy 1.
# Since this is a 2x2 game, the only strategy left would be for both player to play strategy 0.
assert solve_game(example, False) == (0,0)

## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices.

For games that can be solved with *weak* SEDS, there may be more than one solution. You only need to return the first solution found. However, if you would like to return all solutions, you can implement `solve_game` as state space search.

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 3,0 | 2,1 | 1,0 |
|1  | 2,1 | 1,1 | 1,0 |
|2  | 0,1 | 0,1 | 0,0 |

**Solution:**? (strategy indices)

1. For player 1, strategy 0 has strong strict dominance over strategy 2. This means that we should eliminate strategy 2 from the game

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 3,0 | 2,1 | 1,0 |
|1  | 2,1 | 1,1 | 1,0 |
|2  | ~~0,1~~ | ~~0,1~~ | ~~0,0~~ |

2. Next, player 2 goes. For player 2, strategy 1 is strongly dominant over strategy 2. This means that player 2 will not play strategy 2 and it should be eliminated.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 3,0 | 2,1 | ~~1,0~~ |
|1  | 2,1 | 1,1 | ~~1,0~~ |
|2  | ~~0,1~~ | ~~0,1~~ | ~~0,0~~ |

3. Next, pleayer 1 goes. For player 1, strategy 0 is strongly dominante over strategy 1, therefore strategy 1 is eliminated.
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 3,0 | ~~2,1~~ | ~~1,0~~ |
|1  | 2,1 | ~~1,1~~ | ~~1,0~~ |
|2  | ~~0,1~~ | ~~0,1~~ | ~~0,0~~ |

4. Next, player 2 goes. Since 1 > 0, player 2 chooses strategy 1. Therefore strategy 0 is eliminated.
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~3,0~~ | ~~2,1~~ | ~~1,0~~ |
|1  | 2,1 | ~~1,1~~ | ~~1,0~~ |
|2  | ~~0,1~~ | ~~0,1~~ | ~~0,0~~ |

At this point we can identify strategy (0,1) as the winning strategy.

In [12]:
test_game_1 = [[(3,0),(2,1),(1,0)],[(2,1),(1,1),(1,0)],[(0,1),(0,1),(0,0)]]
solution = solve_game(test_game_1, False)
print(solution)

(0, 1)


In [13]:
assert solution == (0,1) # insert your solution from above.

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 2,2 | 3,2 | 3,2.5 |
|1  | 2.5,3 | 2.5,6 | 5,5 |
|2  | 2,3 | 4,4 | 6,2.5 |

**Solution:**? 

1. For player 1, strategy 2 has weak dominance over strategy 0. This means that we should eliminate strategy 0 from the game

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~2,2~~ | ~~3,2~~ | ~~3,2.5~~ |
|1  | 2.5,3 | 2.5,6 | 5,5 |
|2  | 2,3 | 4,4 | 6,2.5 |

2. Next, player 2 goes. For player 2, strategy 1 has weak dominance over strategy 0. This means that we should eliminate strategy 0 from the game

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~2,2~~ | ~~3,2~~ | ~~3,2.5~~ |
|1  | ~~2.5,3~~ | 2.5,6 | 5,5 |
|2  | ~~2,3~~ | 4,4 | 6,2.5 |

3. Next, pleayer 1 goes. For player 1, strategy 2 is weakly dominant over strategy 1, therefore strategy 1 is eliminated.
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~2,2~~ | ~~3,2~~ | ~~3,2.5~~ |
|1  | ~~2.5,3~~ | ~~2.5,6~~ | ~~5,5~~ |
|2  | ~~2,3~~ | 4,4 | 6,2.5 |

4. Next, pleayer 2 goes. For player 1, strategy 1 is weakly dominant over strategy 2, therefore strategy 2 is eliminated.
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~2,2~~ | ~~3,2~~ | ~~3,2.5~~ |
|1  | ~~2.5,3~~ | ~~2.5,6~~ | ~~5,5~~ |
|2  | ~~2,3~~ | 4,4 | ~~6,2.5~~ |

At this point we can identify strategy (2,1) as the winning strategy.

In [14]:
test_game_2 = [[(2,2),(3,2),(3,2.5)],[(2.5,3),(2.5,6),(5,5)],[(2,3),(4,4),(6,2.5)]]

strong_solution = solve_game( test_game_2)
weak_solution = solve_game(test_game_2, weak=True)

In [15]:
assert strong_solution == None
# assert weak_solution == (0,1) # insert your solution from above.

In [16]:
solve_game( test_game_2, weak=True)

(2, 1)

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

The following example is a variation of example 2, what I have done is change a value to ensure that player 1 is worse off at least in a scenario by eliminating strategy 0.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 2,2 | 5,2 | 3,2.5 |
|1  | 2.5,3 | 2.5,6 | 5,6 |
|2  | 2,3 | 4,4 | 6,2.5 |

**Solution:** 
1. Begin by considering player 1 and focus on the second entry of each strategy:
    * Look at the second entry of each strategy:
   - Strategy 1 is not weakly dominant over Strategy 0 because 5>2.5.
   - Strategy 2 is not weakly dominant over Strategy 0 because 5>4.
   * Let's see now if Strategy 0 can be weakly dominant:
   - Strategy 0 is not weakly dominant over Strategy 1 because 2.5>2.
   - Strategy 0 is not weakly dominant over Strategy 2 because 6>3.
   Because not weakly dominant strategy is found, the game carries on as is
   
| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 2,2 | 5,2 | 3,2.5 |
|1  | 2.5,3 | 2.5,6 | 5,6 |
|2  | 2,3 | 4,4 | 6,2.5 |

2. It order to un-stuck the game, we would need then for palyer 2 to be able to eliminate a strategy. We see that Strategy 1 is weakly dominant over Stategy 0. Therefore Strategy 0 is eliminated:

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~2,2~~ | 5,2 | 3,2.5 |
|1  | ~~2.5,3~~ | 2.5,6 | 5,6 |
|2  | ~~2,3~~ | 4,4 | 6,2.5 |

3. Next, player 1 goes. We can see that Strategy 2 weakly dominates Stategy 1, therefore we eliminate Stategy 1:

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | ~~2,2~~ | 5,2 | 3,2.5 |
|1  | ~~2.5,3~~ | ~~2.5,6~~ | ~~5,6~~|
|2  | ~~2,3~~ | 4,4 | 6,2.5 |

At this point we come into a stand still, this is because no Strategy is weakly dominant for either player. The game therefore becomes stuck.  

In [17]:
test_game_3 = [[(2,2),(5,2),(3,2.5)],[(2.5,3),(2.5,6),(5,5)],[(2,3),(4,4),(6,2.5)]]
strong_solution = solve_game(test_game_3)
weak_solution = solve_game(test_game_3, weak=True)

In [18]:
assert strong_solution == None
assert weak_solution == None

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.